In [ ]:
import pandas as pd
import numpy as np
import gensim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report , confusion_matrix

In [ ]:
df = pd.read_csv("clean_data.csv")

In [ ]:
#Apply Mapping to Integer for Label column
df['Label'] = df['Label'].map({'Olahraga':0,'Politik':1,'Hiburan':2})

df.head()

,Unnamed: 0,Teks,Media,Label
0,0,san antonio spurs kalah tuan rumah phoenix sun...,Detik.com,0
1,1,hilang main tottenham hotspur nekat main tahan...,Detik.com,0
2,2,nicolas jackson cetak hat trick menang chelsea...,Detik.com,0
3,3,lionel me i menang ballon d or unggul cukup ja...,Detik.com,0
4,4,liga basket ajar indonesia dbl buat kejut part...,Detik.com,0


In [ ]:
#Split dataset menjadi training dan testing set
X_train, X_test,Y_train, Y_test = train_test_split(df['Teks'],df['Label'],random_state = 42 , test_size=0.2, stratify = df['Label'])

#1. Text Representation 1 : Word2Vec

In [ ]:
#Ambil setiap kata dalam X_train, disini tidak usah tokenization, sebab dalam data cleaning sudah dipastikan bahwa setiap kata terpaut satu spasi
sentences = [sentence.split() for sentence in X_train]

#Train model Word2vec sesuai dengan keinginan soal
#Penjelasan detail ada di pdf
word2vec_model = gensim.models.Word2Vec(sentences,
                                   vector_size=50,
                                   window=10,
                                   min_count=3,
                                   sg = 1)

Disini,
- vector_size menjelaskan mengenai banyaknya vektor dimensi yang akan terbuat dalam proses training nantinya. Disini vector_size = 50, maka nanti setiap data akan memiliki 50 dimensi
- window mengontrol jumlah kata sebelum dan sesudah kata target yang diambil sebagai konteks untuk pelatihan model.Jika window = 10, maka di setiap kata X, akan ada 10 kata sebelum dan sesudah X yang diambil sebagai konteks representative kata tersebut
- min_count menyatakan jumlah kata minimum yang akan diperhitungkan agar kata diperhitungkan dalam model. Jika min_count = 3, maka kata-kata yang muncul kurang dari 3 kali dalam korpus training word2Vec akan diabaikan(valued 0).
- sg = 1 menyatakan mode skip-gram, jika sg = 0 akan menyatakan mode CBOW

In [ ]:
#Lakukan vektorisasi untuk setiap kata
def vectorize_word2vec(sentence): #Parameter = kalimat/teks
    words = sentence.split() #Pecah teks menjadi kata
     #Lakukan word vectorizing untuk setiap kata dari model word2vec yang ada
    words_vecs = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if len(words_vecs) == 0: #Kalau ternyata tidak ada / tidak memenuhi min_count, maka nilainya adalah 0
        return np.zeros(50) #return nilai 0
    words_vecs = np.array(words_vecs) #Kalau tidak 0, jadikan array lalu keluarkan rata-ratanya
    return words_vecs.mean(axis=0)

In [ ]:
#Apply word vectorize ke setiap kata yang ada pada X_train dan X_test
X_train_word2vec= np.array([vectorize_word2vec(sentence) for sentence in X_train])
X_test_word2vec = np.array([vectorize_word2vec(sentence) for sentence in X_test])

# Text Representation 2 : FastText

In [ ]:
#Ambil setiap kata dalam X_train, disini tidak usah tokenization, sebab dalam data cleaning sudah dipastikan bahwa setiap kata terpaut satu spasi
sentences = [sentence.split() for sentence in X_train]

#Train model fastText sesuai dengan keinginan soal
#Penjelasan detail ada di pdf
fastText_model = gensim.models.FastText(sentences,
                                        vector_size=50,
                                        window=10,
                                        min_count=3,
                                        sg=1
                                       )

In [ ]:
#Lakukan vektorisasi untuk setiap kata
def vectorize_fastText(sentence): #Parameter = kalimat/teks
    words = sentence.split() #Pecah teks menjadi kata
    #Lakukan word vectorizing untuk setiap kata dari model word2vec yang ada
    words_vecs = [fastText_model.wv[word] for word in words if word in fastText_model.wv]
    if len(words_vecs) == 0:#Kalau ternyata tidak ada / tidak memenuhi min_count, maka nilainya adalah 0
        return np.zeros(50) #return nilai 0
    words_vecs = np.array(words_vecs) #Kalau tidak 0, jadikan array lalu keluarkan rata-ratanya
    return words_vecs.mean(axis=0)

In [ ]:
#Apply word vectorize ke setiap kata yang ada pada X_train dan X_test
X_train_fastText = np.array([vectorize_fastText(sentence) for sentence in X_train])
X_test_fastText = np.array([vectorize_fastText(sentence) for sentence in X_test])

In [ ]:
X_train_fastText.shape

(360, 50)

# Modelling : SVM

In [ ]:
from sklearn.svm import SVC
svm = SVC()

## 1. Word2Vec embedding

In [ ]:
model_svm_word2vec = svm.fit(X_train_word2vec,Y_train)

prediction_word2vec_svm = model_svm_word2vec.predict(X_test_word2vec)

In [ ]:
print(confusion_matrix(prediction_word2vec_svm, Y_test))

[[30  0  0]
 [ 0 29  1]
 [ 0  1 29]]


In [ ]:
print(classification_report(prediction_word2vec_svm, Y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        30
           1       0.97      0.97      0.97        30
           2       0.97      0.97      0.97        30

    accuracy                           0.98        90
   macro avg       0.98      0.98      0.98        90
weighted avg       0.98      0.98      0.98        90



Disini, terlihat bahwa model SVM bekerja dengan hampir sempurna pada model word2Vec, dimana terdapat
- 1 misklasifikasi kelas 1(Politik) ke kelas 2(Hiburan) dan sebaliknya, mengakibatkan keduanya memiliki precision 97% (29/30 *100%) dan recall 97% juga, sebab hanya 29 prediksi yang benar dari total 30 prediksi label 2(29/30 * 100%).
- Model juga memberikan akurasi akhir 98%

## FastText embedding

In [ ]:
model_svm_fastText = svm.fit(X_train_fastText,Y_train)

prediction_fastText_svm = model_svm_fastText.predict(X_test_fastText)

In [ ]:
print(confusion_matrix(prediction_fastText_svm,Y_test))

[[28  0  0]
 [ 0 29  0]
 [ 2  1 30]]


In [ ]:
print(classification_report(prediction_fastText_svm,Y_test))

              precision    recall  f1-score   support

           0       0.93      1.00      0.97        28
           1       0.97      1.00      0.98        29
           2       1.00      0.91      0.95        33

    accuracy                           0.97        90
   macro avg       0.97      0.97      0.97        90
weighted avg       0.97      0.97      0.97        90



Disini, terlihat bahwa model SVM juga bekerja dengan hampir sempurna pada model fastText, dimana terdapat
- 2 misklasifikasi kelas 0(Olahraga) ke kelas 1(Politik) dan 1 misklasifikasi dari label 1 ke label 2 , mengakibatkan keduanya memiliki precision 93% (28/30 *100%) dan 97%(29/30 * 100%)
- Label 2 memiliki nilai recall 91%, sebab ada 33 prediksi di kelas 2 dan hanya 30 yang benar-benar label 2 (30/33 * 100%).
- Model juga memberikan akurasi akhir 97%

# Modelling : Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

## Word2Vec embedding

In [ ]:
model_rf_word2vec = rf.fit(X_train_word2vec,Y_train)

prediction_word2vec_rf = model_rf_word2vec.predict(X_test_word2vec)

In [ ]:
print(confusion_matrix(prediction_word2vec_rf, Y_test))

[[30  0  2]
 [ 0 30  1]
 [ 0  0 27]]


In [ ]:
print(classification_report(prediction_word2vec_rf, Y_test))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97        32
           1       1.00      0.97      0.98        31
           2       0.90      1.00      0.95        27

    accuracy                           0.97        90
   macro avg       0.97      0.97      0.97        90
weighted avg       0.97      0.97      0.97        90



Pada model random Forest dengan metode word2Vec representation, terlihat bahwa:
- Terdapat 3 misklasifikasi label 2 , 2 data masuk ke dalam Label 0 dan 1 data masuk ke dalam label 1, menghasilkan precision 90% saja (27 / 30 * 100%), sebab hanya 27 data dari 30 data yang terprediksi benar
- Recall pada label 0 dan 1 berakibat menurun menjadi 94%(30/32 * 100%) dan 97%(30/31 *100%) , sebagai akibat dari misklasifikasi label 2
- Akurasi keseluruhan model mencapai 97%

## FastText embedding

In [ ]:
model_rf_fastText = rf.fit(X_train_fastText,Y_train)


prediction_fastText_rf = model_rf_fastText.predict(X_test_fastText)

In [ ]:
print(confusion_matrix(prediction_fastText_rf, Y_test))

[[30  0  1]
 [ 0 28  1]
 [ 0  2 28]]


In [ ]:
print(classification_report(prediction_fastText_rf, Y_test))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98        31
           1       0.93      0.97      0.95        29
           2       0.93      0.93      0.93        30

    accuracy                           0.96        90
   macro avg       0.96      0.96      0.96        90
weighted avg       0.96      0.96      0.96        90



Pada model Random Forest dengan fastText representation, terlihat bahwa
- Label 1 mengalami 2 misklasifikasi ke label 2 , sedangkan label 2 mengalami misklasifikasi ke label 0 dan label 1 sebanyak 1 data masing-masing.
- Berakibat dengan menurunnya precision dan recall masing-masing label
1. Label 0 memiliki recall menjadi 97% (30/31 * 100%)
2. Label 1 memiliki precision 93% (28/30*100%)
3. Label 1 memiliki recall 97% (28/29 * 100%)
4. Label 2 memiliki precision 93% (28/30 * 100%)
5. Label 2 memiliki recall 93% (28/30 * 100%)
- Total akurasi keseluruhan mencapai 96%

# Final Conclusion

Setelah dilakukan scrapping, labelling, kemudian cleaning, text representation dan juga modelling, terlihat bahwa model yang dihasilkan dari setiap text representation sudah sangat baik, dimana total akurasi mencapai >95%.Untuk model terbaik diberikan kepada model SVM dengan text representation word2Vec dengan total akurasi 80%.


Masih banyak hal yang dapat dikulik dari kasus ini, seperti hyperparameter tuning pada kedua model dan juga memodifikasi text representation parameter, terutama min_count. Namun, dikarenakan kendala waktu, hal tersebut masih belum dapat dilakukan